In [1]:
# imports
import nltk
#nltk.download('punkt')  #for first time use download this package
#nltk.download('wordnet') # for first time use
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

# initilizing chatbot training
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


# extract words from intents.json
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        word_tokenize = nltk.word_tokenize(pattern)
        words.extend(word_tokenize)

        # add documents
        documents.append((word_tokenize, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# lemmatize words
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


# initializing training data
training = []
output_empty = [0]*len(classes)
for doc in documents:
    # initializing bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each words (create the base/root words)
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match is found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag 
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# create model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Using TensorFlow backend.


50 documents
13 classes ['Dicussion-room', 'book-prefer', 'borrow-book', 'contact-librarian', 'goodbye', 'greeting', 'greeting-human', 'greeting-name-enquiry', 'help-query', 'issued-book', 'reserved-books', 'search-library management system', 'thanks']
77 unique lemmatized words ["'s", ',', '.', 'a', 'acc', 'according', 'afternoon', 'are', 'awesome', 'book', 'booking', 'borrow', 'bye', 'called', 'can', 'contact', 'discussion', 'do', 'find', 'for', 'from', 'go', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'how', 'i', 'in', 'information', 'is', 'issued', 'later', 'librarian', 'library', "ma'am", 'management', 'me', 'morning', 'my', 'name', 'need', 'nice', 'on', 'prefer', 'preference', 'prefernce', 'query', 'question', 'read', 'reserved', 'room', 'search', 'see', 'should', 'sir', 'some', 'subject', 'system', 'take', 'talking', 'thank', 'thanks', 'that', 'there', 'to', 'ums', 'want', 'what', 'which', 'you', 'your']
Training data created
Epoch 1/200
50/50 

50/50 [==============================] - 0s 658us/step - loss: 0.1749 - accuracy: 0.9400
Epoch 72/200
50/50 [==============================] - 0s 652us/step - loss: 0.2549 - accuracy: 0.9200
Epoch 73/200
50/50 [==============================] - 0s 431us/step - loss: 0.1725 - accuracy: 0.9800
Epoch 74/200
50/50 [==============================] - 0s 202us/step - loss: 0.0827 - accuracy: 1.0000
Epoch 75/200
50/50 [==============================] - 0s 438us/step - loss: 0.0749 - accuracy: 1.0000
Epoch 76/200
50/50 [==============================] - 0s 510us/step - loss: 0.0835 - accuracy: 0.9800
Epoch 77/200
50/50 [==============================] - 0s 864us/step - loss: 0.1283 - accuracy: 0.9800
Epoch 78/200
50/50 [==============================] - 0s 713us/step - loss: 0.0808 - accuracy: 0.9800
Epoch 79/200
50/50 [==============================] - 0s 666us/step - loss: 0.0974 - accuracy: 0.9600
Epoch 80/200
50/50 [==============================] - 0s 687us/step - loss: 0.0443 - accuracy: 

50/50 [==============================] - 0s 433us/step - loss: 0.0587 - accuracy: 0.9600
Epoch 151/200
50/50 [==============================] - 0s 226us/step - loss: 0.0537 - accuracy: 1.0000
Epoch 152/200
50/50 [==============================] - 0s 519us/step - loss: 0.0626 - accuracy: 0.9800
Epoch 153/200
50/50 [==============================] - 0s 674us/step - loss: 0.0492 - accuracy: 0.9800
Epoch 154/200
50/50 [==============================] - 0s 520us/step - loss: 0.0407 - accuracy: 0.9800
Epoch 155/200
50/50 [==============================] - 0s 706us/step - loss: 0.0479 - accuracy: 0.9800
Epoch 156/200
50/50 [==============================] - 0s 681us/step - loss: 0.0626 - accuracy: 1.0000
Epoch 157/200
50/50 [==============================] - 0s 653us/step - loss: 0.0480 - accuracy: 1.0000
Epoch 158/200
50/50 [==============================] - 0s 678us/step - loss: 0.0295 - accuracy: 1.0000
Epoch 159/200
50/50 [==============================] - 0s 652us/step - loss: 0.0201 - a